# ERA5 Plotting Notebook

By: Ty Janoski

## Setup

In [ ]:
# Import statements
import xarray as xr
import scienceplots  # type: ignore
import matplotlib.pyplot as plt
import boto3
from botocore import UNSIGNED
from botocore.config import Config

plt.style.use(["science", "nature"])


In [ ]:
# Make the above code into a function
def read_in_era5_data(bucket_name: str, path: str, hour: str) -> xr.DataArray:
    # Set up S3 client
    s3 = boto3.client(
        "s3", region_name="us-east-1", config=Config(signature_version=UNSIGNED)
    )

    # List objects from AWS Bucket
    response = s3.list_objects_v2(Bucket=bucket_name, Prefix=path)

    # Find key containing hour.grib2.gz, which is the file we want
    for obj in response["Contents"]:
        if f"{hour}.grib2.gz" in obj["Key"]:
            key = obj["Key"]
            break

    # Read in the file
    grib_data = BytesIO()
    s3.download_fileobj(bucket_name, key, grib_data)
    grib_data.seek(0)

    with tempfile.NamedTemporaryFile(suffix=".grib2", delete=False) as tmp_file:
        with gzip.open(grib_data, "rb") as f:
            tmp_file.write(f.read())
        tmp_file_path = tmp_file.name

        
        da = xr.open_dataarray(tmp_file_path, engine="cfgrib")

    return da
    

In [ ]:
bucket_name = "nsf-ncar-era5"

s3 = boto3.client("s3", region_name="us-west-2", config=Config(signature_version=UNSIGNED))

